In [1]:
from web_parser import AnswerParser, QuestionParser
import multiprocessing
from multiprocessing import Queue, Process
from selenium import webdriver
from pathlib import Path
import os
from util.repo import driver_root, database_root, drivers, git_repo_test
from data_handler import Master

In [2]:
q_id = 625769474
# q_id = 460106396
q_id = 327003589
# q_id = 459470265
q_list = [q_id]
config = {
    "simutaneously_scrap_and_store": False,
    "nr_answer_scapers": 2,
    "commiter_name":"test",
    "commiter_email":"test@mal.com"
}
db_path = os.path.join(database_root, "test.2.db")

# TODO
 - [x] test scrap then store
 - [ ] test scrap and store at same time
 - [x] test on a git repo
 - [ ] test to create git repo and make a commit<br>
 https://stackoverflow.com/questions/71947784/what-does-webdriverwaitdriver-20-mean<br>
https://stackoverflow.com/questions/26869200/python-queue-module-get-stuck

---
 - [ ] scrap author
 - [ ] init git repo (partly done)
 - [ ] init db (partly done)
 - [ ] merge repo
 - [ ] revert git operation
 - [ ] scroll
 - [x] git message


In [3]:
class Logger:
    def __init__(self, s:str):
        self.s  = s
        
    def log(self, s):
        print("{}:{}".format(self.s,s))

In [4]:
q_task = Queue()
q_result = Queue()

In [5]:
class TestParser:
    def f(self):
        print("start")
        # return
        i = 9
        while True:
            i-=1
            print(i)
            if i<=0:
                break
                
        print("get_out")
# t = TestParser()
# t.start_parsing()

In [6]:
class Controller:
    def __init__(
        self, 
    ):
        self.processes = []
        pass

In [7]:
question_driver = webdriver.Chrome(drivers["chrome"])
question_parser = QuestionParser(
    queue_put_result=q_result,
    queue_put_task=q_task,
    driver=question_driver,
    logger=Logger("q")
)
answer_drivers = []
answer_parsers = []
for i in range(config["nr_answer_scapers"]):
    answer_drivers.append(
        webdriver.Chrome(drivers["chrome"])
    )
    answer_parsers.append(
        AnswerParser(
            queue_put_result=q_result,
            queue_get_task=q_task,
            driver=answer_drivers[i],
            logger=Logger("a_{}".format(i))
        )
    )
master = Master(
    db_path=db_path,
    git_repo_path=git_repo_test
)


In [8]:
if not config["simutaneously_scrap_and_store"]:
    print("scrap, then store")
    targets = [
        # target func and args
        (question_parser.start_parsing_list, (q_list,))
    ] + [
        (answer_parser.start_parsing, [])
        for answer_parser in answer_parsers
    ] + [
        (master.start, (config["commiter_name"],config["commiter_email"],q_result))
    ]
        
    
    processes = [
        Process(target=target_func, args=args)
        for target_func, args in targets
    ]
    
   
    
        
        
else:
    print(1)

scrap, then store


In [9]:
for p in processes[:-1]:
    p.start()

327003589

q:parsing question with id=327003589a_0:parsing answer with id=3255789284
a_1:parsing answer with id=701487690
q:parsed 327003589
q:fin
a_1:parsing answer with id=728565226
a_0:parsing answer with id=1890558990


In [15]:
for p in processes[:-1]:
    p.start()

# for p in processes:
#     p.join()

327003589
q:parsing question with id=327003589


Process Process-5:
Traceback (most recent call last):
  File "/home/ywatcher/anaconda3/envs/web/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/ywatcher/anaconda3/envs/web/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/media/ywatcher/ExtDisk1/Files/Packages/web_archiv/src/web_parser/question_parser.py", line 94, in start_parsing_list
    self.parse(question_id, single_parse=False)
  File "/media/ywatcher/ExtDisk1/Files/Packages/web_archiv/src/web_parser/question_parser.py", line 32, in parse
    expand_question(self.driver)
  File "/media/ywatcher/ExtDisk1/Files/Packages/web_archiv/src/util/web_util.py", line 22, in expand_question
    all_buttons = driver.find_element_by_xpath(
  File "/home/ywatcher/anaconda3/envs/web/lib/python3.9/site-packages/selenium/webdriver/remote/webelement.py", line 235, in find_element_by_xpath
    return self.find_element(by=By.XPATH, value=x

In [11]:
processes[-1].start()
for p in processes:
    p.join()

In [10]:
# type(l)

search:process queue self._wait_for_tstate_lock() 

In [11]:
from data_handler.master import Master
# from data_handler.database import DataBase
db_path = os.path.join(database_root, "test.2.db")

In [11]:
# warning : another branch

DataBase.init_database(db_file=db_path)
# TODO: move to GitOperor.create_repo
from git import Repo
repo_ = Repo(git_repo_test)
print(repo_.head.object.hexsha)
with DataBase(db_path) as db:
    db.update_git_head(repo_.head.object.hexsha,to_commit=True)

2.6.0
2caed99f0a1d7a82f3f4ef0de9b003b1c2c76a37


In [14]:
results = []
while not q_result.empty():
# for r in q_result:
    r = q_result.get()
    results.append(r)

In [12]:
master = Master(
    db_path=db_path,
    git_repo_path=git_repo_test
)

In [16]:

master.start(
    commiter_name=config["commiter_name"],
    commiter_email=config["commiter_email"],
    result_queue=q_result
)

tzoffset('UTC+0', -28800)
UTC
tzoffset('UTC+0', -28800)
UTC
tzoffset('UTC+0', -28800)
UTC
tzoffset('UTC+0', -28800)
UTC
tzoffset('UTC+0', -28800)
UTC
tzoffset('UTC+0', -28800)
UTC


In [17]:
for p in processes[1:]:
    p.join()
    p.close()